Preparing the dataset

In [1]:
import os
print(os.getcwd())

/home/nb/Documents/GitHub/Codewars/AI


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths 
train_dir = '/home/nb/Documents/GitHub/Codewars/AI/images/train/'
validation_dir = '/home/nb/Documents/GitHub/Codewars/AI/images/validation/'

# Image_data generators
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

2024-07-09 02:15:19.260644: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-09 02:15:19.264484: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-09 02:15:19.277111: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-09 02:15:19.298771: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-09 02:15:19.298812: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-09 02:15:19.312163: I tensorflow/core/platform/cpu_feature_guard.cc:

In [4]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224,224),
                                                    batch_size=32,
                                                    class_mode='binary', # Use 'categorical' if I have more than 2 classes 
                                                ) 

validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                              target_size=(224,224),
                                                              batch_size=32,
                                                              class_mode='binary'
)

Found 3 images belonging to 1 classes.
Found 2 images belonging to 1 classes.


Training the model
- Train a CNN model using TensorFlow

In [5]:
from tensorflow.keras import layers, models

# Define the model 
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Use 'softmax' and change units if you have more than 2 classes
])

# Compile the model 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
try:
# Train the model 
    history = model.fit(train_generator, epochs=10, validation_data=validation_generator)
except ValueError as e :
    print(e)

/home/nb/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/home/nb/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.3333 - loss: 0.6977 - val_accuracy: 1.0000 - val_loss: 6.9380e-13
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 647ms/step - accuracy: 1.0000 - loss: 1.0595e-05 - val_accuracy: 1.0000 - val_loss: 8.6468e-24
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 640ms/step - accuracy: 1.0000 - loss: 7.2209e-10 - val_accuracy: 1.0000 - val_loss: 8.0987e-35
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 644ms/step - accuracy: 1.0000 - loss: 4.1035e-14 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 655ms/step - accuracy: 1.0000 - loss: 1.9698e-18 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 626ms/step - accuracy: 1.0000 - loss: 8.9655e-23 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 634ms/step - accuracy: 1.0000 - loss: 4.3380e-27 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 620ms/step - accuracy: 1.0000 - loss: 2.4

Converting the model to TensorFlow Lite

In [5]:
# Save the model
model.save('terminalia_glaucifolia_craib_model.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model 
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmprmhkhrdy/assets


INFO:tensorflow:Assets written to: /tmp/tmprmhkhrdy/assets


Test the model

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path='model.tflite')
interpreter.allocate_tensors()

# Get input and output tensors.
input_detials = interpreter.get_input_details()
output_detials = interpreter.get_output_details()

In [ ]:
def preprocess_image(img_path, target_size=(224,224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalize the image
    return img_array

In [ ]:
def get_class_label(image_path, class_labels):
    img_array = preprocess_image(image_path)
    interpreter.set_tensor(input_details[0]['index'], img_array)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predicted_class = np.argmax(output_data)
    return class_labels[predicted_class]

In [ ]:
class_labels = {0: 'Terminalia glaucifolia craib', 1: 'class1', 2: 'class2'}

# Test the model with an example image
test_image_path = '/home/nb/Documents/GitHub/Codewars/AI/p6.jpg'
label = get_class_label(test_image_path, class_labels)
print(f"The image is classified as: {label}")